In [1]:
!pip install nsepython

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [4]:
from nsepython import nse_optionchain_scrapper
from datetime import datetime
import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt